## 1. Environment Setup


In [ ]:
# Import libraries
import os
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Dataset Configuration


In [ ]:
# Dataset paths
DATASET_NAME = "Dataset-V1-multiclass"
DATASET_ROOT = os.path.join(os.getcwd(), DATASET_NAME)

# For Google Colab, use environment variable if set
if os.getenv('DATASET_ROOT'):
    DATASET_ROOT = os.getenv('DATASET_ROOT')
    print(f"Using DATASET_ROOT from environment: {DATASET_ROOT}")
else:
    print(f"Using local DATASET_ROOT: {DATASET_ROOT}")

DATA_YAML = os.path.join(DATASET_ROOT, "data.yaml")

# Verify dataset exists
if os.path.exists(DATA_YAML):
    print(f"✅ Dataset found: {DATA_YAML}")
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)
    print(f"Classes: {data_config['names']}")
    print(f"Number of classes: {data_config['nc']}")
else:
    print(f"❌ Dataset not found at: {DATA_YAML}")
    print("Please run tools/create_multiclass_dataset.py first")

## 3. Verify Dataset Statistics


In [ ]:
# Count images in each split
splits = ['train', 'valid', 'test']
dataset_stats = {}

for split in splits:
    images_dir = os.path.join(DATASET_ROOT, split, 'images')
    labels_dir = os.path.join(DATASET_ROOT, split, 'labels')
    
    if os.path.exists(images_dir):
        num_images = len([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
        num_labels = len([f for f in os.listdir(labels_dir) if f.endswith('.txt')])
        dataset_stats[split] = {'images': num_images, 'labels': num_labels}
    else:
        dataset_stats[split] = {'images': 0, 'labels': 0}

print("\n📊 Dataset Statistics:")
print("="*50)
for split, stats in dataset_stats.items():
    print(f"{split.capitalize():8s}: {stats['images']:3d} images, {stats['labels']:3d} labels")
print("="*50)
total_images = sum(s['images'] for s in dataset_stats.values())
print(f"Total:     {total_images:3d} images")

## 4. Visualize Sample Annotations


In [ ]:
def visualize_annotations(image_path, label_path, class_names):
    """
    Visualize YOLO format annotations on image using matplotlib.
    
    Args:
        image_path: Path to image file
        label_path: Path to YOLO format label file
        class_names: Dictionary mapping class IDs to names
    """
    # Read image
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    # Create figure
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    
    # Color map for classes
    colors = {
        0: '#FF0000',  # car - red
        1: '#00FF00',  # lot_boundary - green
        3: '#0000FF',  # stall - blue
    }
    
    # Read and draw annotations
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, bbox_w, bbox_h = map(float, parts[1:5])
                    
                    # Convert YOLO format to pixel coordinates
                    x_center_px = x_center * w
                    y_center_px = y_center * h
                    bbox_w_px = bbox_w * w
                    bbox_h_px = bbox_h * h
                    
                    # Calculate corner coordinates
                    x1 = x_center_px - bbox_w_px / 2
                    y1 = y_center_px - bbox_h_px / 2
                    
                    # Draw rectangle
                    color = colors.get(class_id, '#FFFFFF')
                    rect = plt.Rectangle((x1, y1), bbox_w_px, bbox_h_px,
                                        fill=False, edgecolor=color, linewidth=2)
                    ax.add_patch(rect)
                    
                    # Add label
                    class_name = class_names.get(class_id, f'class_{class_id}')
                    ax.text(x1, y1 - 5, class_name, 
                           color=color, fontsize=10, weight='bold',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
    
    ax.axis('off')
    plt.title(f"Image: {os.path.basename(image_path)}")
    plt.tight_layout()
    plt.show()


# Visualize a sample from train set
train_images = Path(DATASET_ROOT) / 'train' / 'images'
train_labels = Path(DATASET_ROOT) / 'train' / 'labels'

sample_images = list(train_images.glob('*.jpg'))[:3]  # First 3 images

for img_path in sample_images:
    label_path = train_labels / (img_path.stem + '.txt')
    visualize_annotations(img_path, label_path, data_config['names'])

## 5. Training Configuration

Multi-class training with all classes enabled


In [ ]:
# Training configuration
config = {
    # Model
    'model': 'yolo11m.pt',  # YOLOv11 medium pretrained
    
    # Training parameters
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'patience': 20,  # Early stopping
    'workers': 8,
    'device': 0,  # GPU 0, use 'cpu' for CPU training
    
    # Optimizer
    'optimizer': 'AdamW',
    'lr0': 0.00125,  # Initial learning rate
    'lrf': 0.01,     # Final learning rate factor
    'momentum': 0.937,
    'weight_decay': 0.0005,
    
    # Data augmentation
    'hsv_h': 0.015,  # HSV-Hue augmentation
    'hsv_s': 0.7,    # HSV-Saturation augmentation
    'hsv_v': 0.4,    # HSV-Value augmentation
    'degrees': 10.0,  # Rotation degrees
    'translate': 0.1,  # Translation
    'scale': 0.5,     # Scaling
    'shear': 0.0,     # Shear
    'perspective': 0.0,  # Perspective
    'flipud': 0.0,    # Flip up-down
    'fliplr': 0.5,    # Flip left-right
    'mosaic': 1.0,    # Mosaic augmentation
    'mixup': 0.0,     # Mixup augmentation
    
    # Output
    'project': 'parking_runs',
    'name': 'yolo11m_multiclass',
    'exist_ok': True,
    'save': True,
    'save_period': 10,  # Save checkpoint every 10 epochs
}

print("Training Configuration:")
print("="*60)
for key, value in config.items():
    print(f"{key:20s}: {value}")
print("="*60)

## 6. Train Multi-Class Model

Training on car (0), lot_boundary (1), and stall (3) classes


In [ ]:
# Load model
model = YOLO(config['model'])

# Train with all configuration parameters
results = model.train(
    data=DATA_YAML,
    epochs=config['epochs'],
    batch=config['batch'],
    imgsz=config['imgsz'],
    patience=config['patience'],
    workers=config['workers'],
    device=config['device'],
    optimizer=config['optimizer'],
    lr0=config['lr0'],
    lrf=config['lrf'],
    momentum=config['momentum'],
    weight_decay=config['weight_decay'],
    hsv_h=config['hsv_h'],
    hsv_s=config['hsv_s'],
    hsv_v=config['hsv_v'],
    degrees=config['degrees'],
    translate=config['translate'],
    scale=config['scale'],
    shear=config['shear'],
    perspective=config['perspective'],
    flipud=config['flipud'],
    fliplr=config['fliplr'],
    mosaic=config['mosaic'],
    mixup=config['mixup'],
    project=config['project'],
    name=config['name'],
    exist_ok=config['exist_ok'],
    save=config['save'],
    save_period=config['save_period'],
)

print("\n✅ Training completed!")
print(f"Best model saved at: {results.save_dir}/weights/best.pt")

## 7. Visualize Training Results


In [ ]:
# Display results plot
results_img = plt.imread(f"{config['project']}/{config['name']}/results.png")
plt.figure(figsize=(20, 10))
plt.imshow(results_img)
plt.axis('off')
plt.title('Training Results')
plt.tight_layout()
plt.show()

## 8. Load Best Model for Inference


In [ ]:
# Load best model
best_model_path = f"{config['project']}/{config['name']}/weights/best.pt"
model_best = YOLO(best_model_path)

print(f"✅ Loaded best model from: {best_model_path}")

## 9. Validation Metrics


In [ ]:
# Run validation
metrics = model_best.val(data=DATA_YAML)

print("\n📊 Validation Metrics:")
print("="*60)
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print("\nPer-class mAP50:")
for i, (class_name, map50) in enumerate(zip(data_config['names'].values(), metrics.box.maps)):
    print(f"  {class_name:15s}: {map50:.4f}")
print("="*60)

## 10. Confusion Matrix


In [ ]:
# Display confusion matrix
cm_img = plt.imread(f"{config['project']}/{config['name']}/confusion_matrix.png")
plt.figure(figsize=(12, 10))
plt.imshow(cm_img)
plt.axis('off')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

## 11. Test Predictions on Sample Images


In [ ]:
# Get sample test images
test_images_dir = Path(DATASET_ROOT) / 'test' / 'images'
test_images = list(test_images_dir.glob('*.jpg'))[:5]  # First 5 test images

# Run predictions
for img_path in test_images:
    results = model_best.predict(source=str(img_path), save=False, conf=0.25)
    
    # Plot results
    result = results[0]
    img_with_boxes = result.plot()
    img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title(f'Predictions: {img_path.name}')
    plt.tight_layout()
    plt.show()
    
    # Print detection summary
    print(f"\nDetections in {img_path.name}:")
    if len(result.boxes) > 0:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            cls_name = data_config['names'][cls_id]
            print(f"  {cls_name}: {conf:.3f}")
    else:
        print("  No detections")
    print("-" * 50)

## 12. Export Model (Optional)

Export to ONNX, TorchScript, or other formats for deployment


In [ ]:
# Export to ONNX (optional)
# model_best.export(format='onnx', dynamic=True, simplify=True)
# print("✅ Model exported to ONNX format")

print("To export model, uncomment the code above and run this cell")

## Summary

**Next Steps:**

1. Use this trained model to detect cars and stalls in unlabeled images
2. Create occupancy detection pipeline using IoU matching
3. Implement in `occupancy.ipynb` for real-world deployment

**Model outputs:**

- Best weights: `parking_runs/yolo11m_multiclass/weights/best.pt`
- Training results: `parking_runs/yolo11m_multiclass/results.png`
- Confusion matrix: `parking_runs/yolo11m_multiclass/confusion_matrix.png`
